In [2]:
import pandas as pd
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import wget

C:\Users\Zita\AppData\Local\Temp\ipykernel_6084\3133960829.py:3: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
C:\Users\Zita\AppData\Local\Temp\ipykernel_6084\3133960829.py:4: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html


In [3]:
spacex_launch_dash = wget.download("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv")
spacex_df=pd.read_csv(spacex_launch_dash)

In [4]:
spacex_df.head()

,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [66]:
### TASK 1: Add a Launch Site Drop-down Input Component
app = dash.Dash(__name__)

app.layout = html.Div(
    children = [
        html.H1(
            'SpaceX Launch Records Dashboard',
            style = {'textAlign':'center',
                     'color':'#503D36',
                     'font-size':24
                    }
        ),
        ### TASK 1: Add a Launch Site Drop-down Input Component
        dcc.Dropdown(id = 'site-dropdown',
                     options = [
                         {'label':'All Sites', 'value':'ALL'},
                         {'label':'CCAFS LC-40','value':'CCAFS LC-40'},
                         {'label':'CCAFS SLC-40','value':'CCAFS SLC-40'},
                         {'label':'KSC LC-39A','value':'KSC LC-39A'},
                         {'label':'VAFB SLC-4E','value':'VAFB SLC-4E'}
                     ],
                     value = 'ALL',
                     placeholder = 'Select a Launch Site here',
                     searchable = True
            
        ),
        ### TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
        html.Div(dcc.Graph(id = 'success-pie-chart')),
        ### TASK 3: Add a Range Slider to Select Payload
        html.P('Payload range (Kg)'),
        html.Div(dcc.RangeSlider(
            id = 'payload-slider',
            min = 0,
            max = 10000,
            step = 1000,
            #marks = {0 : '0', 100: '100'},
            value = [spacex_df['Payload Mass (kg)'].min(),spacex_df['Payload Mass (kg)'].max()]
        )
                ),
        ### TASK 4
        html.Div(dcc.Graph(id = 'success-payload-scatter-chart'))
    ]
)

### TASK 2: Add a callback function to render success-pie-chart based on selected site dropdown
@app.callback(
    Output(component_id = 'success-pie-chart', component_property = 'figure'),
    Input(component_id = 'site-dropdown', component_property = 'value')
    
)

def get_pie_chart(entered_site):
    filtered_df = spacex_df
    if entered_site == 'ALL':
        fig = px.pie(filtered_df, values = 'class', names = 'Launch Site', title = 'Total Success Launches by Site')
        return fig
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == entered_site]
        filtered_df = filtered_df.groupby(['Launch Site', 'class']).size().reset_index(name = 'numberoflaunches')
        fig = px.pie(filtered_df, values = 'numberoflaunches', names = 'class', title = f'Total Success Launches for {entered_site}')
        return fig
### TASK 4: Add a callback function to render success-payload-scatter-chart scatter plot
@app.callback(
    Output(component_id = 'success-payload-scatter-chart', component_property = 'figure'),
    Input(component_id = 'site-dropdown', component_property = 'value'),
    Input(component_id = 'payload-slider', component_property = 'value')
)

def payload_scatter(entered_site, payload):
    filtered_df = spacex_df[spacex_df['Payload Mass (kg)'].between(payload[0],payload[1])]
    if entered_site == 'ALL':
        fig = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = f'Correlation between Payload and Success for all Sites {payload[0]},{payload[1]}')
        return fig
        print(payload[0],payload[1])
    else:
        filtered_df = filtered_df[filtered_df['Launch Site'] == entered_site]
        fig = px.scatter(filtered_df, x = 'Payload Mass (kg)', y = 'class', color = 'Booster Version Category', title = f"Correlation between Payload and Success for {entered_site}")
        return fig

if __name__ == '__main__':
    app.run_server(debug = True)
